## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing:2px; color:#b57edc; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #b57edc">Libraries</p>

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from tqdm import tqdm_notebook

import warnings
warnings.filterwarnings('ignore')

import re

pd.options.display.max_rows = 1000

ModuleNotFoundError: No module named 'seaborn'

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing:2px; color:#b57edc; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #b57edc">Intro</p>

도쿄의 **치안** 데이터와 **오시마랜드** 데이터를 활용하여 EDA(탐색적 데이터 분석)를 수행하고자 한다.

 <a href = 'http://www.oshimaland.co.jp'> 오시마랜드(Oshimaland)</a>
>This website showcases buildings with unfortunate past incidents such as suicide cases, corpse disposal incidents, and murder cases.

<a href = 'https://catalog.data.metro.tokyo.lg.jp/dataset?q=%E6%B2%BB%E5%AE%89&sort=score+desc%2C+metadata_modified+desc'>일본 도쿄 공공 데이터</a>

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing:2px; color:#b57edc; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #b57edc">EDA</p>

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing:2px; color:#068FFF; font-size:80%; text-align:left;padding: 0px; border-bottom: 3px solid #068FFF">Oshima Jieun Data Processing</p>

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing:2px; color:#1A5D1A; font-size:75%; text-align:left;padding: 0px; border-bottom: 3px solid #1A5D1A">Data info</p>

There are 4 independent variables:
<ul>
<li><strong>oshimaland_dataset_final.csv</strong><ul>
<br>
<li><code>info</code> 사건 정보</li>
<li><code>address</code> 사건이 발생한 주소</li>
<li><code>occurred_date</code> 사건 발생 날짜</li>
<li><code>recorede_date</code> 사건 기록 날짜</li>

In [4]:
oshimaJ= pd.read_csv('C:\\Users\\lucky\\Documents\\COLLABORATION\\AirbnbWise\\Oshimaland_data\\jieun\\oshimaland_dataset_final.csv')
oshimaJ.shape

(570, 4)

In [144]:
oshimaJ.drop(columns = ['recorded_date'], inplace = True) #* recorded_date(기록 날짜) 칼럼 제거
oshimaJ.rename(columns = {'occurred_date': 'date'}, inplace = True) #* occured_date -> date 칼럼명 변경

In [145]:
#* address 칼럼 가공
#! 일본어 주소는 일반적으로 '시' 또는 '구', '동/읍/면', '번지', 그리고 '건물명' 등의 정보로 구성
#* (1) address의 모든 데이터에 '東京都(도쿄도)' 포함되어 있으므로 제거
oshimaJ['address'] = oshimaJ['address'].str.replace('東京都','')
oshimaJ.tail()

,info,address,date
565,死体発見,武蔵野市西久保三丁目75都築荘101,平成28年2月23日
566,5階 角部屋\r\n告知事項有り,国分寺市戸倉一丁目825 PlaceK,令和4年10月6日
567,6階 角部屋\r\n告知事項有り,国分寺市戸倉一丁目825 PlaceK,令和4年10月6日
568,206号室、506号室\r\n告知事項あり,国分寺市戸倉一丁目825 プレイスK,令和4年7月30日
569,606号室\r\n告知事項あり,国分寺市戸倉一丁目825 プレイスK,令和4年7月30日


In [146]:
#* (2) '시(市) / 구(区) 칼럼(shiku) 추가'
#* oshimaJ['address'].str.contains('市').sum(), 주소에 市(시)가 포함되어 있는 주소는 186개
#* 주소에 市(시)가 없다면 구(区)로 추출
def districtExtract(address):
    if '市' in address:
        extracted_data = address[:address.index('市') + 1]
    elif '市' not in address and '区' in address:
        extracted_data = address[:address.index('区') + 1]
    else: 
        extracted_data = address #* '外国外国板橋二丁目639' 데이터는 시, 구가 모두 없는 유일한 데이터로 삭제해준다.
    return extracted_data

oshimaJ['shiku'] = oshimaJ['address'].apply(districtExtract)

drop_index = oshimaJ[oshimaJ['shiku'] == '外国外国板橋二丁目639'].index
oshimaJ = oshimaJ.drop(drop_index).reset_index(drop=True)
display(oshimaJ.head(), oshimaJ.tail())

,info,address,date,shiku
0,819号室で女性死亡,新宿区歌舞伎町一丁目202,平成30年5月23日,新宿区
1,飛び降り自殺,新宿区歌舞伎町一丁目202アパホテル新宿歌舞伎町タワー↓,令和3年5月18日,新宿区
2,告知事項あり\r\n※管理会社に直接確認済み\r\n\r\nハイム大成ビル 403号室\r\...,新宿区西新宿七丁目1813,1年以内,新宿区
3,１４Ｆから女性が飛び降り自殺,新宿区西新宿七丁目1713パレステュディオ新宿WEST,令和元年9月23日,新宿区
4,事故死,新宿区西新宿七丁目2231,2016,新宿区


,info,address,date,shiku
564,死体発見,武蔵野市西久保三丁目75都築荘101,平成28年2月23日,武蔵野市
565,5階 角部屋\r\n告知事項有り,国分寺市戸倉一丁目825 PlaceK,令和4年10月6日,国分寺市
566,6階 角部屋\r\n告知事項有り,国分寺市戸倉一丁目825 PlaceK,令和4年10月6日,国分寺市
567,206号室、506号室\r\n告知事項あり,国分寺市戸倉一丁目825 プレイスK,令和4年7月30日,国分寺市
568,606号室\r\n告知事項あり,国分寺市戸倉一丁目825 プレイスK,令和4年7月30日,国分寺市


In [147]:
#* (3) town(동/ 읍/ 면) 칼럼 추가 -> town 구역 + 건물 번호 모두 추출
shikuList = list(oshimaJ['shiku'])
def townExtract(address):
    if isinstance(address, str):
        for idx in range(len(shikuList)):
            if shikuList[idx] in address:
                find_data = address[address.find(shikuList[idx]) + len(shikuList[idx]):]
                return find_data

oshimaJ['town'] = oshimaJ['address'].apply(townExtract)
display(oshimaJ.head(), oshimaJ.tail())


,info,address,date,shiku,town
0,819号室で女性死亡,新宿区歌舞伎町一丁目202,平成30年5月23日,新宿区,歌舞伎町一丁目202
1,飛び降り自殺,新宿区歌舞伎町一丁目202アパホテル新宿歌舞伎町タワー↓,令和3年5月18日,新宿区,歌舞伎町一丁目202アパホテル新宿歌舞伎町タワー↓
2,告知事項あり\r\n※管理会社に直接確認済み\r\n\r\nハイム大成ビル 403号室\r\...,新宿区西新宿七丁目1813,1年以内,新宿区,西新宿七丁目1813
3,１４Ｆから女性が飛び降り自殺,新宿区西新宿七丁目1713パレステュディオ新宿WEST,令和元年9月23日,新宿区,西新宿七丁目1713パレステュディオ新宿WEST
4,事故死,新宿区西新宿七丁目2231,2016,新宿区,西新宿七丁目2231


,info,address,date,shiku,town
564,死体発見,武蔵野市西久保三丁目75都築荘101,平成28年2月23日,武蔵野市,西久保三丁目75都築荘101
565,5階 角部屋\r\n告知事項有り,国分寺市戸倉一丁目825 PlaceK,令和4年10月6日,国分寺市,戸倉一丁目825 PlaceK
566,6階 角部屋\r\n告知事項有り,国分寺市戸倉一丁目825 PlaceK,令和4年10月6日,国分寺市,戸倉一丁目825 PlaceK
567,206号室、506号室\r\n告知事項あり,国分寺市戸倉一丁目825 プレイスK,令和4年7月30日,国分寺市,戸倉一丁目825 プレイスK
568,606号室\r\n告知事項あり,国分寺市戸倉一丁目825 プレイスK,令和4年7月30日,国分寺市,戸倉一丁目825 プレイスK


In [148]:
#* (4) 'town' 내에서 구역이 존재하는 데이터 들의 구역 추출하여 칼럼 추가
#* oshimaJ['town'].str.contains('丁目').sum(), '丁目' 이 존재하는 데이터는 542개, 존재하지 않는 데이터는 26개
def partExtract(town):
    if '丁目' in town:
        extracted_data = town[:town.find('丁目')] + '丁目'
    else:
        extracted_data = np.nan #* 구역 없음
    return extracted_data

oshimaJ['townpart'] = oshimaJ['town'].apply(partExtract)

In [149]:
#* (5) 'town'내에서 구역이 존재하지 않는 데이터들의 상세 주소 추출하여 칼럼 추가
def detailExtract(town):
    if '丁目' not in town:
        extracted_data = town
    else:
        extracted_data = town[town.find('丁目') + 2:]
    return extracted_data 

oshimaJ['detailpart'] = oshimaJ['town'].apply(detailExtract)

In [150]:
#* oshimaJ['town'].str.contains('丁目').sum(), '丁目' 이 존재하는 데이터는 542개
#* '丁目'이 존재하지 않는 경우, 주소에 숫자가 포함되어 있다면 숫자 이전 문자열 까지 구역으로 추출 하고, 숫자가 없다면 그대로 추출
#! 숫자가 두개 이상 포함된 경우가 있으므로, 이 함수는 적용하지 않는다.
# def partExtract(town):
#     if '丁目' in town:
#         extracted_data = town[:town.find('丁目')] + '丁目'
#     else:
#         for idx, char in enumerate(town):
#             if any(char.isdigit() for char in town): #* town 주소에서 하나 이상의 숫자가 발견되었는지 확인
#                 match = re.search(r'\d+', town)
#                 extracted_data = town[:match.start()]
#             else:
#                 extracted_data = town[idx]
#     return extracted_data

#oshimaJ['townpart'] = oshimaJ['town'].apply(partExtract)

In [151]:
#* (4) 특수 문자 제거 -> 이 데이터의 경우 '\r', '\n', '↓' 를 제거해야 한다.
def remove_special_characters(text):
    if isinstance(text, str):  #* 문자열인 경우에만 정규표현식 적용
        pattern = r'[\r\n↓]'
        cleaned_text = re.sub(pattern, '', text)
        return cleaned_text
    return text

oshimaJ_cleaned = oshimaJ.applymap(remove_special_characters)

In [152]:
oshimaJ_cleaned.shape

(569, 7)

In [153]:
#* (5) 칼럼 순서 변경
oshimaJ_cleaned = oshimaJ_cleaned.reindex(columns = ['info', 'date', 'address', 'shiku', 'town', 'townpart', 'detailpart'])

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing:2px; color:#068FFF; font-size:80%; text-align:left;padding: 0px; border-bottom: 3px solid #068FFF">Oshima Yunyoung Data Processing</p>

In [154]:
oshimaY = pd.read_csv('C:\\Users\\lucky\\Documents\\COLLABORATION\\AirbnbWise\\Oshimaland_data\\yunyoung\\selenium_tokyo_data.csv', index_col = 0)
oshimaY.head()

,district,date,address,content
0,新宿区,平成24年11月5日,東京都新宿区新宿三丁目22-7指田ビル地下1階,死体発見
1,新宿区,平成24年11月5日,東京都新宿区新宿三丁目22-7指田ビル地下1階,死体発見
2,新宿区,令和4年,東京都新宿区西新宿一丁目3-9,飛び降り自殺
3,新宿区,NaN,NaN,NaN
4,新宿区,平成20年1月5日,東京都新宿区歌舞伎町一丁目27-5 中台ビル,2階サウナロイヤル 死体発見


In [155]:
#! 일본어 주소는 일반적으로 '시' 또는 '구', '동/읍/면', '번지', 그리고 '건물명' 등의 정보로 구성
#* (1) address의 모든 데이터에 '東京都(도쿄도)' 포함되어 있으므로 제거
oshimaY['address'] = oshimaY['address'].str.replace('東京都','')

In [156]:
oshimaY.rename(columns = {'content': 'info', 'district' : 'shiku'}, inplace = True) #* content -> info 칼럼명 변경
oshimaY = oshimaY.reindex(columns = ['info', 'date','address', 'shiku'])
oshimaY.tail()

,info,date,address,shiku
298,駐車スペース横の部屋で住民男性が首吊り自殺,2005,羽村市緑ヶ丘五丁目1-50,羽村市
299,告知事項,不明,羽村市羽中一丁目3-37,武蔵村山市
300,死体発見,平成24年,武蔵村山市中央一丁目24-3ハウスオブブーケA 1階,昭島市
301,火災による死亡,令和2年9月7日,昭島市田中町一丁目34-9,福生市
302,通路から駐車場へ男性が飛び下り自殺,2016年,福生市志茂156-1,福生市


In [157]:
#* oshimaJ_cleaned 데이터프레임에 맞춰 oshimaY 데이터프레임 변경
display(oshimaJ_cleaned.head(1), oshimaY.tail())

,info,date,address,shiku,town,townpart,detailpart
0,819号室で女性死亡,平成30年5月23日,新宿区歌舞伎町一丁目202,新宿区,歌舞伎町一丁目202,歌舞伎町一丁目,202


,info,date,address,shiku
298,駐車スペース横の部屋で住民男性が首吊り自殺,2005,羽村市緑ヶ丘五丁目1-50,羽村市
299,告知事項,不明,羽村市羽中一丁目3-37,武蔵村山市
300,死体発見,平成24年,武蔵村山市中央一丁目24-3ハウスオブブーケA 1階,昭島市
301,火災による死亡,令和2年9月7日,昭島市田中町一丁目34-9,福生市
302,通路から駐車場へ男性が飛び下り自殺,2016年,福生市志茂156-1,福生市


In [158]:
#* (2) 'town' 칼럼 추가
shikuList = list(oshimaY['shiku'])
oshimaY['town'] = oshimaY['address'].apply(townExtract)

In [159]:
#* (3) 'town' 칼럼 결측치를 처리해야 다음 코드에서 에러가 나지 않는다.
nullNum = oshimaY['town'].isnull().sum()
print(f'기존 결측치 개수 : {nullNum}개')
oshimaY['town'] = oshimaY['town'].fillna('Not data')

기존 결측치 개수 : 13개


In [160]:
#* (4) 'town' 내에서 구역이 존재하는 데이터 들의 구역 추출하여 칼럼 추가
oshimaY['townpart'] = oshimaY['town'].apply(partExtract)

In [161]:
#* (5) 'town'내에서 구역이 존재하지 않는 데이터들의 상세 주소 추출하여 칼럼 추가
oshimaY['detailpart'] = oshimaY['town'].apply(detailExtract)

In [162]:
#* (6) 특수문자 제거 
oshimaY_cleaned = oshimaY.applymap(remove_special_characters)

In [163]:
#* (7) oshimaY 데이터 중 None 값을 NaN으로 대체
oshimaY_cleaned = oshimaY_cleaned.applymap(lambda x: np.nan if x is None else x)

In [164]:
#* (8) oshimaY town 관련 데이터 중 'Not data' 값을 NaN으로 대체
oshimaY_cleaned = oshimaY_cleaned.applymap(lambda x: np.nan if x == 'Not data' else x)
oshimaY_cleaned.head()

,info,date,address,shiku,town,townpart,detailpart
0,死体発見,平成24年11月5日,新宿区新宿三丁目22-7指田ビル地下1階,新宿区,新宿三丁目22-7指田ビル地下1階,新宿三丁目,22-7指田ビル地下1階
1,死体発見,平成24年11月5日,新宿区新宿三丁目22-7指田ビル地下1階,新宿区,新宿三丁目22-7指田ビル地下1階,新宿三丁目,22-7指田ビル地下1階
2,飛び降り自殺,令和4年,新宿区西新宿一丁目3-9,新宿区,西新宿一丁目3-9,西新宿一丁目,3-9
3,NaN,NaN,NaN,新宿区,NaN,NaN,NaN
4,2階サウナロイヤル 死体発見,平成20年1月5日,新宿区歌舞伎町一丁目27-5 中台ビル,新宿区,歌舞伎町一丁目27-5 中台ビル,歌舞伎町一丁目,27-5 中台ビル


## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing:2px; color:#068FFF; font-size:80%; text-align:left;padding: 0px; border-bottom: 3px solid #068FFF">Data 탐색</p>

In [165]:
#* 각 데이터 shape
print(oshimaJ_cleaned.shape, oshimaY_cleaned.shape)

(569, 7) (303, 7)


In [166]:
#* 각 데이터 결측치 확인
print(oshimaJ_cleaned.isnull().sum())
print('-' * 15)
print(oshimaY_cleaned.isnull().sum())

info           0
date           3
address        0
shiku          0
town           0
townpart      28
detailpart     0
dtype: int64
---------------
info          17
date          20
address       13
shiku          0
town          13
townpart      32
detailpart    13
dtype: int64


In [167]:
#* 각 데이터 중복데이터 확인
print(oshimaJ_cleaned.duplicated().sum(), oshimaY_cleaned.duplicated().sum())

4 44


In [168]:
#* 두 데이터 병합
oshima = pd.concat([oshimaJ_cleaned, oshimaY_cleaned]).reset_index(drop = True)
print(oshima.shape)

(872, 7)


In [169]:
#* town 칼럼 삭제
oshima = oshima.drop(columns = ['town'])

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing:2px; color:#1A5D1A; font-size:75%; text-align:left;padding: 0px; border-bottom: 3px solid #1A5D1A">Data info</p>

There are 6 independent variables:
<ul>
<li><strong>oshima</strong><ul>
<br>
<li><code>info</code> 사건 정보</li>
<li><code>date</code> 사건 발생 날짜</li>
<li><code>address</code> 사건이 발생한 주소</li>
<li><code>shiku</code> 시/구</li>
<li><code>townpart</code> 동/읍/면</li>
<li><code>detailpart</code> 상세 주소</li>

In [170]:
#* 중복 데이터 제거
oshima = oshima.drop_duplicates()
oshima = oshima.reset_index(drop=True)
print(oshima.shape)

(818, 6)


In [171]:
oshima.isnull().sum() #! 결측치 처리 고민...

info          10
date          15
address        6
shiku          0
townpart      51
detailpart     6
dtype: int64

In [172]:
oshima.to_csv('oshimaEDA.csv', index = False)

In [173]:
oshima.head()

,info,date,address,shiku,townpart,detailpart
0,819号室で女性死亡,平成30年5月23日,新宿区歌舞伎町一丁目202,新宿区,歌舞伎町一丁目,202
1,飛び降り自殺,令和3年5月18日,新宿区歌舞伎町一丁目202アパホテル新宿歌舞伎町タワー,新宿区,歌舞伎町一丁目,202アパホテル新宿歌舞伎町タワー
2,告知事項あり※管理会社に直接確認済みハイム大成ビル 403号室賃料¥78000 管理費¥20...,1年以内,新宿区西新宿七丁目1813,新宿区,西新宿七丁目,1813
3,１４Ｆから女性が飛び降り自殺,令和元年9月23日,新宿区西新宿七丁目1713パレステュディオ新宿WEST,新宿区,西新宿七丁目,1713パレステュディオ新宿WEST
4,事故死,2016,新宿区西新宿七丁目2231,新宿区,西新宿七丁目,2231


## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing:2px; color:#068FFF; font-size:80%; text-align:left;padding: 0px; border-bottom: 3px solid #068FFF">Tokyo Open Data Processing</p>

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing:2px; color:#1A5D1A; font-size:75%; text-align:left;padding: 0px; border-bottom: 3px solid #1A5D1A">Data info</p>

There are 20 independent variables:
<ul>
<li><strong>TokyoCrime.csv</strong><ul>
<br>
<li><code>罪名</code> 범죄의 종류 또는 범죄명</li>
<li><code>手口</code> 범죄의 접근 방식 또는 행동</li>
<li><code>管轄警察署（発生地）</code> 범죄가 발생한 지역의 관할 경찰서</li>
<li><code>管轄交番・駐在所（発生地）</code> 범죄가 발생한 지역의 관할 파출소 또는 장애소</li>
<li><code>市区町村コード（発生地）</code> 범죄가 발생한 지역의 시, 구, 동 등을 식별하는 코드</li>
<li><code>都道府県（発生地）</code> 범죄가 발생한 지역의 도 또는 특별시</li>
<li><code>市区町村（発生地）</code> 범죄가 발생한 지역의 시, 구, 동</li>
<li><code>町丁目（発生地）</code> 범죄가 발생한 지역의 동 등 자세한 정보</li>
<li><code>発生時（始期）</code> 범죄가 발생한 날짜</li>
<li><code>発生年月日（始期）</code> 범죄가 발생한 시간</li>
<li><code>発生場所の属性</code> 범죄가 발생한 장소의 특성</li>
<li><code>現金以外の主な被害品</code> 현금 이외의 주요 피해 물품</li>
<li><code>被害者の性別</code> 피해자의 성별</li>
<li><code>被害者の年齢</code>  피해자의 나이</li>
<li><code>現金被害の有無</code> 현금 피해의 여부</li>
<li><code>施錠関係</code> 잠금 관련 정보</li>
<li><code>盗難防止装置の有無</code> 도난 방지 장치의 여부</li>
<li><code>被害者の職業</code> 피해자의 직업</li>
<li><code>発生場所</code> 범죄가 발생한 장소</li>
<li><code>発生場所の詳細</code> 범죄가 발생한 장소에 대한 자세한 정보</li>


In [174]:
openData = pd.read_csv('C:\\Users\\lucky\\Documents\\COLLABORATION\\AirbnbWise\\Tokyo_OpenData\\Integration of downloaded CSV files\\TokyoCrime.csv',  encoding='SHIFT_JIS', index_col=0)
openData.shape

(132320, 20)

In [175]:
#* : 칼럼명을 일본어에서 영어로 변경
openData.columns = ['crimeType', 'criminalBehavior', 'policeStation', 'policeboxOrsubstation','crimeCode', 
                     'crimeAddressSido', 'crimeAddressDong', 'crimeDetailAddress', 'dateOfCrime', 'timeOfCrime', 'crimeSceneCharacteristics', 
                     'otherThanCash', 'genderOfVictim', 'ageOfVictim', 'cashDamagePresence', 'lockingInfo','preventionDevicePresence',
                     'jobOfVictim', 'crimeScene', 'crimeDetailinfo']

In [176]:
openData.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 132320 entries, 0 to 132319
Data columns (total 20 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   crimeType                  132320 non-null  object 
 1   criminalBehavior           132320 non-null  object 
 2   policeStation              132320 non-null  object 
 3   policeboxOrsubstation      132240 non-null  object 
 4   crimeCode                  132281 non-null  float64
 5   crimeAddressSido           132288 non-null  object 
 6   crimeAddressDong           132288 non-null  object 
 7   crimeDetailAddress         132247 non-null  object 
 8   dateOfCrime                132320 non-null  object 
 9   timeOfCrime                132320 non-null  object 
 10  crimeSceneCharacteristics  80634 non-null   object 
 11  otherThanCash              11039 non-null   object 
 12  genderOfVictim             683 non-null     object 
 13  ageOfVictim                11

In [177]:
#* 칼럼 중 '被害者の性別'(genderOfVictim) - 전체 데이터에서 0.5%
# '盗難防止装置の有無'(preventionDevicePresence) - (전체 데이터의 3%)
#  두 칼럼은 압도적으로 데이터가 적으므로 삭제한다. 
#TODO '現金以外の主な被害品'(otherThanCash 11039 Not Null) 칼럼은 전체 데이터의 8%, '現金被害の有無'(cashDamagePresence 11200 Not Null) 칼럼도 전체 데이터의 8% 이다. 이 두 칼럼은 삭제할지 말지 고민이다.
openData = openData.drop(columns = ['genderOfVictim', 'preventionDevicePresence'])
openData.shape

(132320, 18)

In [178]:
# TODO: 'timeOfCrime' 칼럼 데이터 시간 양식이 동일하지 않다. 양식을 통일 하고, 데이터 타입을 datetime(ns)로 변경한다.
# ex) 2023-07-18 , 2023/07/18
# 추후 year, month, day 칼럼으로 분리하여 추가할 예정이다.
def getDateOfCrime(date):
    if '-' in date:
        dateValue = pd.to_datetime(date, format='%Y-%m-%d')
    elif '/' in date:
        dateValue = pd.to_datetime(date, format='%Y-%m-%d')
    else: 
        dateValue = pd.to_datetime(date, errors = 'coerce') #* 날짜가 잘못된 경우는 Nat(Not a Time)으로 처리
    return dateValue

openData['dateOfCrime'] = openData['dateOfCrime'].apply(getDateOfCrime)

In [179]:
# TODO : 이 데이터에서 'crimeType' 즉, 범죄 유형은 '窃盗'(절도)만 존재 하므로, 'crimeType' 칼럼은 삭제한다.
print(openData['crimeType'].value_counts())
openData = openData.drop(columns = ['crimeType'])

窃盗    132320
Name: crimeType, dtype: int64


In [180]:
# TODO 범죄 주소 시/도 + 동/읍/면 + 상세 주소 텍스트를 합쳐서 전체 주소 칼럼을 만들어서, oshimaland 데이터의 주소값과 비교할 수 있도록 한다.
openData['crimeFullAddress'] = openData['crimeAddressSido'] + openData['crimeAddressDong'] + openData['crimeDetailAddress']
display(oshima.head(), display(openData.head()) )
#* oshima 데이터의 'townpart' 칼럼과 openData 데이터의 'crimeDetailAddrss' 칼럼 모두, 구역을 표현한다.

,criminalBehavior,policeStation,policeboxOrsubstation,crimeCode,crimeAddressSido,crimeAddressDong,crimeDetailAddress,dateOfCrime,timeOfCrime,crimeSceneCharacteristics,otherThanCash,ageOfVictim,cashDamagePresence,lockingInfo,jobOfVictim,crimeScene,crimeDetailinfo,crimeFullAddress
0,部品ねらい,中央,本町交番,131024.0,東京都,中央区,日本橋本石町４丁目,2018-03-25,13,その他,ナンバープレート,NaN,NaN,NaN,NaN,NaN,NaN,東京都中央区日本橋本石町４丁目
1,部品ねらい,中央,日本橋交番,131024.0,東京都,中央区,日本橋１丁目,2018-01-22,09,道路上,その他,NaN,NaN,NaN,NaN,NaN,NaN,東京都中央区日本橋１丁目
2,部品ねらい,中央,日本橋交番,131024.0,東京都,中央区,日本橋室町１丁目,2018-05-01,15,道路上,その他,NaN,NaN,NaN,NaN,NaN,NaN,東京都中央区日本橋室町１丁目
3,部品ねらい,中央,八重洲通交番,131024.0,東京都,中央区,八重洲１丁目,2017-12-20,不明,道路上,ナンバープレート,NaN,NaN,NaN,NaN,NaN,NaN,東京都中央区八重洲１丁目
4,部品ねらい,中央,八重洲通交番,131024.0,東京都,中央区,八重洲２丁目,2018-11-09,11,駐車（輪）場,その他,NaN,NaN,NaN,NaN,NaN,NaN,東京都中央区八重洲２丁目


,info,date,address,shiku,townpart,detailpart
0,819号室で女性死亡,平成30年5月23日,新宿区歌舞伎町一丁目202,新宿区,歌舞伎町一丁目,202
1,飛び降り自殺,令和3年5月18日,新宿区歌舞伎町一丁目202アパホテル新宿歌舞伎町タワー,新宿区,歌舞伎町一丁目,202アパホテル新宿歌舞伎町タワー
2,告知事項あり※管理会社に直接確認済みハイム大成ビル 403号室賃料¥78000 管理費¥20...,1年以内,新宿区西新宿七丁目1813,新宿区,西新宿七丁目,1813
3,１４Ｆから女性が飛び降り自殺,令和元年9月23日,新宿区西新宿七丁目1713パレステュディオ新宿WEST,新宿区,西新宿七丁目,1713パレステュディオ新宿WEST
4,事故死,2016,新宿区西新宿七丁目2231,新宿区,西新宿七丁目,2231


None

In [181]:
#TODO 숫자를 제외한 일본어 문자열 삭제, 60대는 하나의 value로 통일
#* 法人・団体、被害者なし - 법인 / 단체가 관련되었으나 피해자 없음의 내용으로 이 값이 들어간 데이터는 제거 한다.
openData['ageOfVictim'].value_counts()

20歳代           28685
10歳代           23837
30歳代           18404
40歳代           15980
50歳代           10557
70歳以上           5867
60-64歳          2865
65-69歳          2794
法人・団体、被害者なし      957
10歳未満            818
Name: ageOfVictim, dtype: int64

In [182]:
# 'ageOfVictim' 칼럼에서 "法人・団体、被害者なし" 값을 np.nan으로 바꿔줌
openData['ageOfVictim'] = openData['ageOfVictim'].replace("法人・団体、被害者なし", '0')

In [183]:
#* \D는 숫자가 아닌 모든 문자를 의미
#! 歳代 ?세대, 歳以上 ?세 이상, 歳未満 ?세 미만
def ageProcessing(age):
    if '-' in age:
        age = '60' #* 60-64, 65-69를 60대로 통일
    else:
        age = re.sub(r'\D', '', age)
    return age

openData['ageOfVictim'] = openData['ageOfVictim'].apply(lambda x: np.nan if pd.isnull(x) else ageProcessing(x))  #! 10은 10세 미만

In [184]:
#TODO timeOfCrime 칼럼의 '不明' 값의 의미는 미확인이다. 이 값을 어떻게 변경할지 고민중이다.